In [1]:
# Cài thư viện cần (dựa trên code trong repo)
!pip install transformers torch datasets accelerate

In [2]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 118.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 94.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 113.4 MB/s eta 0:00:0000:0100:01
     ━━━━

In [3]:
# Kiểm tra GPU có sẵn không
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

# Cài lại các thư viện cần (nếu chưa chạy trước đó)
!pip install transformers torch datasets accelerate pandas scikit-learn tqdm

# Nếu bạn đã ở trong thư mục repo thì bỏ dòng này
%cd /content/Big-Five-personality-score-predictions_RoBERTa_and_BERT  # thay đổi nếu cần

GPU available: True
GPU name: Tesla T4
[Errno 2] No such file or directory: '/content/Big-Five-personality-score-predictions_RoBERTa_and_BERT # thay đổi nếu cần'
/kaggle/working


In [4]:
# Đường dẫn đến file trong folder WevMining (thay tên file nếu khác)
file_path = "/kaggle/input/dataset/pandora_train.csv"  # hoặc .parquet nếu là Parquet

import pandas as pd

# Load file từ Drive
df = pd.read_csv(file_path)  # Nếu là Parquet: pd.read_parquet(file_path)
df = df.sample(frac=0.1, random_state=42)


print("Cột gốc trong file:", df.columns.tolist())

# Rename cột để khớp với code/repo mong đợi (body cho text)
df = df.rename(columns={
    'text': 'body',
    'A': 'agr',
    'O': 'ope',
    'C': 'con',
    'E': 'ext',
    'N': 'neu'
})

# Kiểm tra sau rename
print("\nCột sau khi rename:", df.columns.tolist())
print("\nSample 3 dòng đầu:")
print(df.head(3))

Cột gốc trong file: ['O', 'C', 'E', 'A', 'N', 'ptype', 'text', '__index_level_0__']

Cột sau khi rename: ['ope', 'con', 'ext', 'agr', 'neu', 'ptype', 'body', '__index_level_0__']

Sample 3 dòng đầu:
          ope   con   ext   agr   neu  ptype  \
258181    2.0  38.0  87.0  28.0   1.0      4   
768973   81.0  80.0  30.0  24.0  75.0     25   
1732080  29.0  40.0   6.0   7.0  98.0      1   

                                                      body  __index_level_0__  
258181   People who know they're close to zero balance ...            1386091  
768973                        Aww You are most welcome. =]            2044374  
1732080            Oh, didn't know it blocked her ult. TIL            1512060  


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.optim import AdamW
from tqdm import tqdm
import os

# Load dataset đã rename
df[['ext', 'neu', 'agr', 'con', 'ope']] = df[['ext', 'neu', 'agr', 'con', 'ope']]/100

print("Dataset ready. Shape:", df.shape)

# Hiển thị nội dung của roberta_large_pandora.py để so sánh
print("\n--- Nội dung của roberta_large_pandora.py ---")
!cat roberta_large_pandora.py
print("------------------------------------------")

2026-01-19 08:11:29.164042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768810289.380788      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768810289.447655      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768810290.024109      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768810290.024168      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768810290.024171      55 computation_placer.cc:177] computation placer alr

Dataset ready. Shape: (192420, 8)

--- Nội dung của roberta_large_pandora.py ---
cat: roberta_large_pandora.py: No such file or directory
------------------------------------------


In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

max_length = 128  # Tăng lên 128 để capture tốt hơn (repo gốc 64 quá ngắn cho Reddit comment)

def tokenize_text(text):
    return tokenizer.encode_plus(
        str(text),
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors=None
    )

df['body'] = df['body'].apply(tokenize_text)

print("Tokenization completed. Sample:", df['body'].iloc[0]['input_ids'][:10])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Tokenization completed. Sample: [0, 4763, 54, 216, 51, 214, 593, 7, 4276, 2394]


In [7]:
# Drop author nếu còn (dataset HF thường không có)
if 'author' in df.columns:
    df.drop(['author'], axis=1, inplace=True)

df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

print(f"Train: {len(df_train)} | Val: {len(df_val)} | Test: {len(df_test)}")

Train: 107755 | Val: 26939 | Test: 57726


In [8]:
class PersonalityDataset(Dataset):
    def __init__(self, tweets, targets):
        self.tweets = tweets
        self.targets = targets

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.tweets[idx]['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(self.tweets[idx]['attention_mask'], dtype=torch.long),
            'targets': torch.tensor(self.targets[idx], dtype=torch.float)
        }

In [9]:
# batch_size = 8  # Giảm nếu Colab T4 hết VRAM (8-16 OK cho roberta-large)

# train_dataset = PersonalityDataset(df_train['body'].tolist(), df_train[['ext', 'neu', 'agr', 'con', 'ope']].values)
# val_dataset = PersonalityDataset(df_val['body'].tolist(), df_val[['ext', 'neu', 'agr', 'con', 'ope']].values)
# test_dataset = PersonalityDataset(df_test['body'].tolist(), df_test[['ext', 'neu', 'agr', 'con', 'ope']].values)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

batch_size = 6           # Giảm xuống để tránh OOM với roberta-large

# Chỉ lấy cột 'ope' (Openness)
train_dataset = PersonalityDataset(
    df_train['body'].tolist(),
    df_train[['ope']].values          # ← chỉ 1 cột thay vì 5
)

val_dataset = PersonalityDataset(
    df_val['body'].tolist(),
    df_val[['ope']].values
)

test_dataset = PersonalityDataset(
    df_test['body'].tolist(),
    df_test[['ope']].values
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Dataset chỉ cho Openness (O) đã sẵn sàng. Train samples: {len(train_dataset)}")

Dataset chỉ cho Openness (O) đã sẵn sàng. Train samples: 107755


In [10]:
class RoBERTaForPersonalityTraits(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-large')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(1024, 1)  # 5 traits

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        output = self.dropout(pooled)
        return self.linear(output)

model = RoBERTaForPersonalityTraits()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss()

print("Model loaded on:", device)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on: cuda


In [12]:
from transformers import RobertaModel

class RoBERTaForOpenness(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-large')
        self.dropout = torch.nn.Dropout(0.35)
        self.linear  = torch.nn.Linear(1024, 1)          # Chỉ 1 output cho Openness

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        output = self.dropout(pooled)
        return self.linear(output).squeeze(-1)           # Trả về scalar (batch_size,)

In [13]:
# epochs =20   # Bắt đầu với 3 để test, sau tăng
# patience = 3
# best_val_loss = float('inf')
# patience_counter = 0
# accumulation_steps = 4  # Gradient accumulation để tiết kiệm VRAM

# log_file = 'roberta_large_results.txt'

# with open(log_file, 'w') as f:
#     f.write("Training started\n")

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         step = 0

#         train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/Training")
#         for i, batch in enumerate(train_bar):
#             if i % accumulation_steps == 0:
#                 optimizer.zero_grad()

#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             targets = batch['targets'].to(device)

#             outputs = model(input_ids, attention_mask)
#             loss = loss_fn(outputs, targets)

#             loss = loss / accumulation_steps
#             loss.backward()

#             if (i + 1) % accumulation_steps == 0 or i + 1 == len(train_loader):
#                 optimizer.step()
#                 optimizer.zero_grad()

#             total_loss += loss.item() * accumulation_steps
#             step += 1

#             train_bar.set_postfix({'loss': total_loss / step})

#             # Checkpoint
#             if step % 1000 == 0:
#                 torch.save(model.state_dict(), f'checkpoint_step_{step}.pth')

#         avg_train_loss = total_loss / step
#         f.write(f"Epoch {epoch+1}: Train Loss {avg_train_loss}\n")

#         # Validation
#         model.eval()
#         val_loss = 0
#         preds, trues = [], []
#         with torch.no_grad():
#             for batch in tqdm(val_loader, desc="Validation"):
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 targets = batch['targets'].to(device)

#                 outputs = model(input_ids, attention_mask)
#                 val_loss += loss_fn(outputs, targets).item()

#                 preds.append(outputs.cpu().numpy())
#                 trues.append(targets.cpu().numpy())

#         avg_val_loss = val_loss / len(val_loader)
#         f.write(f"Val Loss: {avg_val_loss}\n")

#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             patience_counter = 0
#             torch.save(model.state_dict(), 'best_model.pth')
#         else:
#             patience_counter += 1
#             if patience_counter >= patience:
#                 f.write("Early stopping\n")
#                 break

#     torch.save(model.state_dict(), 'final_model.pth')
#     f.write("Training done\n")

model = RoBERTaForOpenness()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=6e-6)   # lr nhỏ hơn tí cho an toàn
loss_fn = torch.nn.MSELoss()

epochs = 15
patience = 4
best_val_loss = float('inf')
patience_counter = 0
accumulation_steps = 8

log_file = 'roberta_large_openness_only.txt'

with open(log_file, 'w') as f:
    f.write("Training chỉ cho Openness (O)\n")
    f.write("Started\n\n")

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        step = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for i, batch in enumerate(train_bar):
            if i % accumulation_steps == 0:
                optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device).squeeze(-1)   # ← squeeze rất quan trọng!

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, targets)

            loss = loss / accumulation_steps
            loss.backward()

            if (i + 1) % accumulation_steps == 0 or i + 1 == len(train_loader):
                optimizer.step()
                optimizer.zero_grad()

            total_loss += loss.item() * accumulation_steps
            step += 1
            train_bar.set_postfix({'loss': f"{total_loss / step:.5f}"})

        avg_train_loss = total_loss / step
        f.write(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.6f}\n")

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc="Validation")
            for batch in val_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                targets = batch['targets'].to(device).squeeze(-1)

                outputs = model(input_ids, attention_mask)
                val_loss += loss_fn(outputs, targets).item()

        avg_val_loss = val_loss / len(val_loader)
        f.write(f"Val Loss: {avg_val_loss:.6f}\n")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model_openness.pth')
            f.write("→ Saved best model\n")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                f.write("Early stopping\n")
                break

    torch.save(model.state_dict(), 'final_model_openness.pth')
    f.write("Training done\n")

print("Training cho Openness hoàn tất.")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 5:  26%|██▋       | 4750/17960 [38:50<1:48:00,  2.04it/s, loss=0.08905]


KeyboardInterrupt: 

In [15]:
# Load model tốt nhất
model.load_state_dict(torch.load('best_model_openness.pth'))
model.eval()

test_loss = 0
all_preds = []
all_trues = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Test Openness"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device).squeeze(-1)

        outputs = model(input_ids, attention_mask)
        test_loss += loss_fn(outputs, targets).item()

        all_preds.extend(outputs.cpu().numpy())
        all_trues.extend(targets.cpu().numpy())

mse = mean_squared_error(all_trues, all_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(all_trues, all_preds)
r2 = r2_score(all_trues, all_preds)

print("\nKết quả trên test set - Openness (O):")
print(f"MSE:  {mse:.5f}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE:  {mae:.5f}")
print(f"R²:   {r2:.5f}")

/tmp/ipykernel_55/3628600644.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_openness.pth'))
Test Openness: 100%|██████████|


Kết quả trên test set - Openness (O):
MSE:  0.09076
RMSE: 0.30126
MAE:  0.26226
R²:   0.06866
